# Demo 5: Collaborative Filtering and Comedy! 
------
<img src="images/seinfeld.jpg" width="400" height="400">

#### Real Dataset: http://eigentaste.berkeley.edu/dataset/ Dataset 2 
#### Rate Jokes: http://eigentaste.berkeley.edu

## What are we trying to learn from this dataset?

# QUESTION:  Can Collaborative Filtering be used to find which jokes to recommend to our users?


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas
import cassandra
import pyspark
import re
import os
import matplotlib.pyplot as plt
from IPython.display import IFrame
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

#### Helper function to have nicer formatting of Spark DataFrames

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  10, truncate = False):
    if(truncate):
        pandas.set_option('display.max_colwidth', 100)
    else:
        pandas.set_option('display.max_colwidth', None)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

## Creating Tables and Loading Tables

### Connect to Cassandra

In [4]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    'secure_connect_bundle': '/home/jovyan/secureconnect/secure-connect-colvin-better-botz.zip'
}
auth_provider = PlainTextAuthProvider(username='james', password='@DSjwc127!')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

### Create Demo Keyspace 

In [ ]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS accelerate 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

### Set keyspace 

In [5]:
session.set_keyspace('betterbotz')

### Create table called jokes. Our PRIMARY will need to be a unique composite key (userid, jokeid). This will result in an even distribution of the data and allow for each row to be unique. Remember we will have to utilize that PRIMARY KEY in our WHERE clause in any of our CQL queries. 

In [ ]:
query = "CREATE TABLE IF NOT EXISTS jokes \
                                    (userid int, jokeid int, rating float, \
                                     PRIMARY KEY (userid, jokeid))"
session.execute(query)

### What do these of these 3 columns represent: 

* **Column 1**: User id
* **Column 2**: Joke id
* **Column 3**: Rating of joke (-10.00 - 10.00) 

### Load Jokes dataset from CSV file (jester_ratings3.csv)
* This is a file I created from the *.dat file and I only have 10,000 rows -- dataset has over 1 million rows
<img src="images/laughing.gif" width="300" height="300">

#### Insert all the Joke Rating Data into the table `jokes`

In [ ]:
fileName = 'data/jester_ratings3.csv'
input_file = open(fileName, 'r')

for line in input_file:
    jokeRow = line.split(',')
    query = "INSERT INTO jokes (userid, jokeid, rating)"
    
    query = query + "VALUES (%s, %s, %s)"
    
    session.execute(query, (int(jokeRow[0]), int(jokeRow[1]) , float(jokeRow[2]) ))

#### Do a select * on joke_table WHERE userid = x to verify that data was loaded into the table

In [6]:
query = 'SELECT * FROM jokes WHERE userid = 65'
rows = session.execute(query)
for row in rows:
    print (row.userid, row.jokeid, row.rating)

65 5 9.875
65 7 9.718999862670898
65 8 10.0
65 13 8.625
65 15 7.406000137329102
65 16 -9.875
65 17 -9.531000137329102
65 18 -8.5
65 19 -7.156000137329102
65 20 -0.09399999678134918
65 21 6.938000202178955
65 22 10.0
65 23 9.937999725341797
65 24 6.843999862670898
65 25 10.0
65 26 9.968999862670898
65 27 3.2809998989105225
65 28 9.968999862670898
65 29 2.0309998989105225
65 30 10.0
65 31 6.938000202178955
65 32 2.937999963760376
65 33 10.0
65 34 10.0
65 35 5.75
65 36 2.875
65 37 10.0
65 38 10.0
65 39 9.937999725341797
65 40 9.968999862670898
65 41 10.0
65 42 9.968999862670898
65 43 10.0
65 44 9.875
65 45 9.968999862670898
65 46 9.812000274658203
65 47 10.0
65 48 6.938000202178955
65 49 6.938000202178955
65 50 1.0
65 51 10.0
65 52 9.968999862670898
65 53 6.0
65 54 6.938000202178955
65 55 9.937999725341797
65 56 10.0
65 57 9.875
65 58 9.875
65 59 10.0
65 60 10.0
65 61 6.938000202178955
65 62 6.938000202178955
65 63 9.937999725341797
65 64 9.937999725341797
65 65 6.938000202178955
65 66 6.

<img src="images/sparklogo.png" width="150" height="200">

### Finally time for Apache Spark! 

#### Create a spark session that is connected to Cassandra. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [8]:
spark = SparkSession \
    .builder \
    .appName('demo') \
    .master("local") \
    .config( \
        "spark.cassandra.connection.config.cloud.path", \
        "file:/home/jovyan/secureconnect/secure-connect-colvin-better-botz.zip") \
    .config("spark.cassandra.auth.username", "james") \
    .config("spark.cassandra.auth.password", "@DSjwc127!") \
    .getOrCreate()

jokeTable = spark.read.format("org.apache.spark.sql.cassandra").options(table="jokes", keyspace="betterbotz").load()

print ("Table Row Count: ")
print (jokeTable.count())

Table Row Count: 
10000


#### Split dataset into training and testing set 

In [9]:
(training, test) = jokeTable.randomSplit([0.8, 0.2])

training_df = training.withColumn("rating", training.rating.cast('int'))
testing_df = test.withColumn("rating", test.rating.cast('int'))

showDF(training_df)

,userid,jokeid,rating
0,5,5,6
1,5,7,4
2,5,8,-5
3,5,13,0
4,5,15,-4
5,5,16,3
6,5,17,6
7,5,18,5
8,5,19,6
9,5,20,5


### Setup for CFliter with ALS

https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

In [10]:
als = ALS(maxIter=5, regParam=0.01, userCol="userid", itemCol="jokeid", ratingCol="rating",
          coldStartStrategy="drop")

model = als.fit(training_df)

In [11]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(testing_df)

# Generate top 10 joke recommendations for each user
userRecs = model.recommendForAllUsers(10)

showDF(userRecs)

# Generate top 10 user recommendations for each joke
jokeRecs = model.recommendForAllItems(10)

,userid,recommendations
0,148,"[(97, 30.287826538085938), (87, 25.366596221923828), (120, 25.031978607177734), (76, 24.890043258666992), (104, 23.9307918548584), (114, 22.42287826538086), (53, 21.929218292236328), (95, 21.208690643310547), (118, 21.16971206665039), (37, 20.66012191772461)]"
1,243,"[(120, 32.379642486572266), (41, 26.605640411376953), (37, 24.12421417236328), (71, 24.06833267211914), (138, 22.878767013549805), (125, 18.292133331298828), (146, 14.526366233825684), (27, 13.850459098815918), (58, 10.646146774291992), (87, 10.283836364746094)]"
2,251,"[(55, 13.173197746276855), (146, 13.0884370803833), (139, 12.635540962219238), (141, 7.326050758361816), (143, 2.9512789249420166), (138, 2.7054619789123535), (28, 1.7319763898849487), (59, 1.7102030515670776), (120, 1.5420844554901123), (134, 0.14325737953186035)]"
3,85,"[(33, 56.57527542114258), (73, 52.55316162109375), (39, 50.38872146606445), (105, 38.27677536010742), (60, 37.8847541809082), (74, 37.725341796875), (23, 37.699310302734375), (92, 36.5486946105957), (86, 36.41018295288086), (94, 36.00797653198242)]"
4,137,"[(82, 11.839689254760742), (103, 11.297845840454102), (7, 10.585664749145508), (100, 9.444416046142578), (85, 9.115898132324219), (44, 8.99427604675293), (8, 8.983870506286621), (145, 8.967763900756836), (21, 8.89068603515625), (71, 8.724923133850098)]"
5,65,"[(44, 12.585005760192871), (73, 11.871543884277344), (99, 11.64207935333252), (145, 11.426669120788574), (22, 11.040337562561035), (85, 11.000626564025879), (64, 10.954109191894531), (60, 10.921162605285645), (38, 10.692020416259766), (78, 10.53303050994873)]"
6,53,"[(146, 15.670921325683594), (33, 13.63193416595459), (38, 13.101275444030762), (46, 11.774742126464844), (110, 11.241178512573242), (23, 10.615825653076172), (25, 10.598196029663086), (60, 10.187507629394531), (128, 9.966042518615723), (105, 9.20401382446289)]"
7,133,"[(83, 11.999852180480957), (75, 11.122471809387207), (80, 10.408059120178223), (97, 9.939044952392578), (119, 9.043347358703613), (69, 8.9954833984375), (35, 8.953094482421875), (31, 8.839223861694336), (81, 8.43713092803955), (121, 8.344532012939453)]"
8,155,"[(103, 73.35855102539062), (44, 55.93171691894531), (121, 46.63677215576172), (55, 45.512474060058594), (85, 42.45634841918945), (51, 41.21355056762695), (79, 39.89340591430664), (78, 39.81708526611328), (88, 38.932743072509766), (100, 38.74076843261719)]"
9,108,"[(138, 27.454126358032227), (52, 22.09364891052246), (123, 19.40737533569336), (124, 18.9996337890625), (58, 17.746326446533203), (82, 16.416767120361328), (139, 15.855134010314941), (57, 15.353630065917969), (51, 9.520898818969727), (86, 9.35458755493164)]"


In [12]:
showDF(userRecs.filter(userRecs.userid == 65))

,userid,recommendations
0,65,"[(44, 12.585005760192871), (73, 11.871543884277344), (99, 11.64207935333252), (145, 11.426669120788574), (22, 11.040337562561035), (85, 11.000626564025879), (64, 10.954109191894531), (60, 10.921162605285645), (38, 10.692020416259766), (78, 10.53303050994873)]"


In [13]:
IFrame(src='images/init94.html', width=700, height=200)

In [14]:
IFrame(src='images/init43.html', width=700, height=200)

In [16]:
spark.stop()